In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import string
import re
from deep_translator import GoogleTranslator
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from langchain.prompts import PromptTemplate
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
import sys

sys.path.append(os.path.abspath(".."))
from src.funcs import translate_by_division
from src.funcs import extract_sentences


#### filter negative annoted sentences using gpt4o

In [2]:
import os
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

In [ ]:
df = pd.read_csv('../data/processed/review_data_berlin_en.csv')

In [ ]:
cols = ['namenr', 'place_ids', 'lat', 'lon', 'geometry',
       'review_text']

In [ ]:
df = df[cols]

In [ ]:
df['review_en'] = df['review_text']df.apply(lambda x: translate_by_division(x))

##### prompt eng

In [ ]:
prompt = PromptTemplate.from_template(
    '''
    Your task is to extract sentences from {sentence}.
    You are an assistant for labeling negative annotaded sentences in user reviews.
    Return the sentences that are associated to be negative in form of a list.
    Provide output without further text information.
    Use the following schema ['sentence 1', 'sentence 2', ...]
    '''
)

In [ ]:
df['sentences'] = df['review_en'].apply(lambda x: extract_sentences(prompt=x, client=OpenAI()))

In [ ]:
all_sentences = []

for row in df.sentences:
    cleaned_row = row.replace("[", "").replace("]", "").replace("\\n", "").strip()
    sentences = cleaned_row.split(", '")
    all_sentences.extend(sentences)

In [ ]:
docs = [sentence for sentence in all_sentences if sentence.lower() != "json"]

#### bert modelling